In [1]:
import pandas as pd
import re
import os

In [2]:
# current working directory -> cwd
cwd = os.getcwd()
def getDir(location):
  for x in os.listdir(location):
      if x.endswith(".xlsx"):
          # Prints only xlsx file present in My Folder
          return x
src = getDir(cwd)

name_of_file = 'LatLongGeocodes.xlsx'

xls = pd.read_excel('{0}\{1}'.format(cwd,src))

try:
    os.makedirs("Output")
except FileExistsError:
    # directory already exists
    pass

In [3]:
colnames = list(xls.columns.values)
addressIndex = None
#df1 = pd.read_excel(xls, 'Address')
for cols in colnames:
    if re.search('Address|address|location|Location', cols):
        addressIndex = colnames.index(cols)
        break
    else:
        continue

In [4]:
xls['{0}'.format(colnames[addressIndex])] = xls['{0}'.format(colnames[addressIndex])].astype(str)
Address = colnames[addressIndex]
address_list = xls['{0}'.format(Address)].tolist() #address in a list
address_list = list(dict.fromkeys(address_list))
no_of_address = len(address_list)
pincodes = []
smalldigPincodes = []

# Modifications

In [5]:
def removeHash(address):
    #removes # and all attached characters from address

    return re.sub(r'(?:\#\s.?[\w]\S+)','',address)

def removeCOetc(address):
    # removes slash and all attached characters from address

    return re.sub(r"^(?:S/O|D/O|W/O|C/O)\s(?:[A-Za-z]+\s?)+,\s(.*)",'',address)

def removeSlash(address):
    # removes slash and all attached characters from address

    return re.sub(r'(?:[/][^\s]+)', '', address)

def removeLandmark(address):
    # removes all after Land mark

    return re.sub(r'(?:[Ll]andmark|[Ll]and mark)\s+', '', address)

def removeBlock(address):
    # remove Block

    return re.sub(r'(?:\w+\s+[bB][lL]ock?[kK])\s+', '', address)

def removeMostNo(address):
    # removes flat no. 2030 and Room No 945

    address = re.sub(r'(?:^|\b)[Nn][Oo]\.?(?:\b|\s+)\d+', '', address)
    address = re.sub(r'(?:^|\b)(?:[fF]lat\.?|[Ff][Ll][Tt]\.?)(?:\b|\s+)', '', address)
    address = re.sub(r'(?:^|\b)[Dd]oor [Nn]o\.?|[Dd]oor [Nn]um\.?|[Dd]oor [Nn]umber\.?(?:\b|\s+)\d+', '', address)
    return re.sub(r'[,/]\d+', '', address)


def removeTrailing(address):
    # remove trailing ,:. or newline character with double whitespace (so that white space gets trimmed)

    return re.sub(r'\W+$','  ',address)

def removeLeading(address):
    # remove leading ,:. or newline character with double whitespace (so that white space gets trimmed)

    return re.sub(r'^\W+','  ',address)

def removeLeadingNum(address):
    # remove leading (number), eg 37, *insert address here*

    return re.sub(r'^\d+[,\s]*', '', address)

def removeMoreLeadingNum(address):
    # remove more leading (number) eg 37 *insert address here*

    return re.sub(r'^\d+\W*', '', address)


def removePhMobNumber(address):
    # remove leading ,:. or newline character with double whitespace (so that white space gets trimmed)

    return re.sub(r'(?:Ph|Mob|ph|mob)[:\s]*[#:]?.?([0-9]{10})', '', address)


def removeFloor(address):
    # remove floor

    return re.sub(r'\b[0-9]{1,3}(?:st|nd|rd|th)?\b[-\s]*(?:floor|Floor|FLR|flr|Flr|FLOOR)','  ',address)

def removePincodes(address):
    # remove pincodes, source trust me bro

    return re.sub(r'([1-9][0-9]{5})','  ',address)

def retainPincodes(address,index):
    # store pincodes

    pin = re.findall(r'([1-9][0-9]{5})',address)
    pincodes.append([pin, index])

def removePost(address):
    # remove post due to map misdirection

    return re.sub(r'([pP]ost)','  ',address)

def removeHouseBuildingStreetNo(address):
    # remove house building number

    return re.sub(r'^\d+\s*(?:,|-| |/)', '', address)

def removeColonySectorPhase(address):
    # remove colony, sector, phase

    return re.sub(r'\b(colony|sector|phase|Colony|Sector|Phase|PHASE|COLONY|SECTOR)\b', '', address)

def removePostOffice(address):
    # remove pose office

    return re.sub(r'\b(Post Office|PO)\b', '', address)

def removePOBox(address):
    # remove PO Box

    return re.sub(r'\b(PO Box)\b', '', address)

def removeVia(address):
    # remove Via or off

    return re.sub(r'\b(via)|(off)\b', '', address)

def removeMainRoad(address):
    # remove Main Road

    return re.sub(r'\d*\s*(?:\d+(?:st|nd|rd|th|ST|ND|RD|TH)\s+)?[Mm]ain\s[Rr]oad\s*,\s*', '', address)

def removeCross(address):
    # Remove some cross

    return re.sub(r'\d*\s*(?:\d+(?:st|nd|rd|th|ST|ND|RD|TH)\s+)?[a-zA-Z]+\s[Cc]ross\s*,\s*', '', address)


def removeState(address):
    # remove state

    states = ['Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chhattisgarh', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jharkhand', 'Karnataka', 'Kerala', 'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha', 'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana', 'Tripura', 'Uttarakhand', 'Uttar Pradesh', 'West Bengal']
    for state in states:
        address = re.sub(r'\b'+state+'\b', '', address)
    return address


def lastDigitsOfPin(address,index):
    # save all pincodes less than 6 digit at the end of string

    pin = re.findall(r'([0-9]{1,3}$)',address)
    smalldigPincodes.append([pin,index])
            
def removeLastDigitsOfPin(address):
    # remove pincodes less than 6 digit at the end of string

    return re.sub(r'([0-9]{1,3}$)','  ',address)

def appendPinCodes(address,index):

    pin1 = (pincodes[index][0])
    if pin1 != []:
        address = address + ' ' + pin1[0]
    else:
        pass
    pin2 = (smalldigPincodes[index][0])
    if pin2 != []:
        address = address + ' ' + pin2[0]
    else:
        pass
    return address


def clean_address(address):
    address = removeHash(address)
    address = removeSlash(address)
    address = removeCOetc(address)
    address = removeMainRoad(address)
    address = removeCross(address)
    address = removeFloor(address)
    address = removeBlock(address)
    address = removeHouseBuildingStreetNo(address)
    address = removeColonySectorPhase(address)
    address = removePostOffice(address)
    address = removePOBox(address)
    address = removeLandmark(address)
    address = removeVia(address)
    address = removeState(address)
    address = removePincodes(address)
    address = removePhMobNumber(address)
    address = removeLeadingNum(address)
    address = removeMoreLeadingNum(address)
    address = removeLeading(address)
    return address

# Main

In [6]:
geocodes = []
address = []

for idx, loc in enumerate(address_list):
    loc = loc.replace(u'\xa0', u' ')
    loc = re.sub(' +', ' ', loc)
    loc = loc.replace(",",", ").replace(".",". ")
    retainPincodes(loc,idx)
    loc = clean_address(loc)
    lastDigitsOfPin(loc,idx)
    loc = appendPinCodes(loc,idx)
    loc = removeMostNo(loc)
    loc = removeLeadingNum(loc)
    loc = removeMoreLeadingNum(loc)
    loc = removeLeading(loc)
    loc = re.sub(' +', ' ', loc)
    loc = loc.strip()
    address.append(loc) 


In [7]:
# dictionary of lists  
dict = dict = {'Original': address_list,'Cleaned': address}  
       
df = pd.DataFrame(dict)

if os.path.exists(('{0}\Output\{1}').format(cwd,name_of_file)):
    os.remove(('{0}\Output\{1}').format(cwd,name_of_file))
else:
    print("Can not delete the file as it doesn't exists")

df.to_excel(
        '{}\Output\{}'.format(cwd, name_of_file), sheet_name="Sheet_1", index=False)